In [1]:
import cv2
import numpy as np
import random


In [2]:
def add_gaussian_noise(image, mean=0, stddev=10):
    """
    Add Gaussian noise to an image.

    Parameters:
        image (numpy.ndarray): Input image.
        mean (float): Mean of the Gaussian noise.
        stddev (float): Standard deviation of the Gaussian noise.

    Returns:
        numpy.ndarray: Image with added Gaussian noise.
    """
    noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
    noisy_image = image.astype(np.float32) + noise
    noisy_image = np.clip(noisy_image, 0, 255)  # Ensure pixel values are in valid range
    return noisy_image.astype(image.dtype)

In [3]:
def generate_star_layer_s(image_shape, num_stars, max_radius, sigma=1):
    """Generate a star layer with a Gaussian distribution."""
    height, width = image_shape
    stars_layer = np.zeros((height, width), dtype=np.uint8)
    mask = np.zeros((height, width), dtype=np.uint8)


    # Loop to place each star
    for _ in range(num_stars):
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        I0 = random.random() * 0.6 + 0.01


        # Gaussian function to create a star with Gaussian distribution
        for i in range(-max_radius, max_radius + 1):
            for j in range(-max_radius, max_radius + 1):
                # Calculate the distance from the center (x, y)
                r = np.sqrt(i**2 + j**2)
                
                if r <= max_radius:
                    # Gaussian formula to calculate the intensity
                    intensity = I0 * np.exp(-(r**2) / (2 * sigma**2)) * 255
                    # Ensure that the pixel is within bounds
                    nx, ny = x + i, y + j
                    if 0 <= nx < width and 0 <= ny < height:
                        stars_layer[ny, nx] = min(255, stars_layer[ny, nx] + intensity)
                        mask[ny, nx] = 1

    return stars_layer, mask

In [4]:

# def generate_star_layer(image_shape, num_stars=200, max_radius=2):
#     """Generate a grayscale star layer matching the input image shape."""
#     height, width = image_shape
#     stars_layer = np.zeros((height, width), dtype=np.uint8)

#     for _ in range(num_stars):
#         x = random.randint(0, width - 1)
#         y = random.randint(0, height - 1)
#         radius = random.randint(1, max_radius)
#         brightness = int(min(255, 100 + radius * 50))  # Adjust brightness based on radius
#         # brightness = random.randint(140, 255)  # star brightness

#         cv2.circle(stars_layer, (x, y), radius, brightness, -1, lineType=cv2.LINE_AA)
#         if radius > 4:
#             # Add a halo effect for larger stars
#             halo_radius = radius + random.randint(1, 3)
#             halo_brightness = max(0, brightness - random.randint(50, 100))
#             cv2.circle(stars_layer, (x, y), halo_radius, halo_brightness, -1, lineType=cv2.LINE_AA)

#     return stars_layer


def generate_star_layer_m(image_shape, num_stars, max_radius, sigma=2.0):
    """Generate a star layer with a Gaussian distribution."""
    height, width = image_shape
    stars_layer = np.zeros((height, width), dtype=np.uint8)
    mask = np.zeros((height, width), dtype=np.uint8)

    # Loop to place each star
    for _ in range(num_stars):
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        I0 = random.random() * 1 + 0.5

        # Gaussian function to create a star with Gaussian distribution
        for i in range(-max_radius, max_radius + 1):
            for j in range(-max_radius, max_radius + 1):
                # Calculate the distance from the center (x, y)
                r = np.sqrt(i**2 + j**2)
                
                if r <= max_radius:
                    # Gaussian formula to calculate the intensity
                    intensity = I0 * np.exp(-(r**2) / (2 * sigma**2)) * 255
                    # Ensure that the pixel is within bounds
                    nx, ny = x + i, y + j
                    if 0 <= nx < width and 0 <= ny < height:
                        stars_layer[ny, nx] = min(255, stars_layer[ny, nx] + intensity)
                        mask[ny, nx] = 1  # Mark the position of the star in the mask

    return stars_layer, mask



def add_stars_to_image(name, input_path, output_path, num_stars=[200, 300], max_radius=[3, 8], noise = True, rescaling = True):
    """Add fake stars to a grayscale image and save the result."""
    # name = input_path.split("/")[-1].split(".")[0]
    image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    # Rescale the image to reduce brightness
    if rescaling:
        image = cv2.convertScaleAbs(image, alpha=0.4, beta=0)
    if image is None:
        raise FileNotFoundError(f"Could not load image from path: {input_path}")

    cv2.imwrite(output_path + "/backgrounds/" + name + ".png", image)

    stars, mask1 = generate_star_layer_m(image.shape, num_stars=num_stars[1], max_radius = max_radius[1])

    result = cv2.add(image, stars)
    stars, mask2 = generate_star_layer_s(image.shape, num_stars=num_stars[0], max_radius = max_radius[0])
    result = cv2.add(result, stars)
    mask = cv2.bitwise_or(mask1, mask2) * 255
    if noise:
        result = add_gaussian_noise(result, mean = 0, stddev = 8)
    cv2.imwrite(output_path + "/stretched/" + name + ".png", result)
    cv2.imwrite(output_path + "/masks/" + name + ".png", mask)
    return result, mask




In [5]:

# === USAGE ===
# Replace with your actual image path:
name = "i1"
input_path = "backgrounds/IMG_2816.jpg"
output_path = "generated"
result, mask = add_stars_to_image(name = name, input_path = input_path, output_path = output_path, num_stars=[1500, 600])

In [6]:
# === USAGE ===
# Replace with your actual image path:
name = "i2"
input_path = "backgrounds/IMG_2833.jpg"
output_path = "generated"
result, mask = add_stars_to_image(name = name, input_path = input_path, output_path = output_path, num_stars=[1500, 600])

In [7]:
# === USAGE ===
# Replace with your actual image path:
name = "i3"
input_path = "backgrounds/IMG_2816.jpg"
output_path = "generated"
result, mask = add_stars_to_image(name = name, input_path = input_path, output_path = output_path, num_stars=[1500, 600], rescaling= False)

In [8]:
# === USAGE ===
# Replace with your actual image path:
name = "i4"
input_path = "backgrounds/IMG_2780.jpg"
output_path = "generated"
result, mask = add_stars_to_image(name = name, input_path = input_path, output_path = output_path, num_stars=[1500, 600], rescaling= False)

In [ ]:
# # Just to check the similarity of stars

# output_path = "generated/m1.png"
# result = add_stars_to_image(input_path, output_path, num_stars=[1000, 400], noise = False, rescaling = False)

In [ ]:
import os

# ------------------------------------------------------------------
# 4) CREATE STRETCHED-MASKED IMAGES
# ------------------------------------------------------------------
# we assume every mask file has a matching stretched file (same name)
dst_root_msk = "masks"
dst_root_str = "stretched"
dst_root_smk = "stretched-masked"

mask_filenames = [f for f in os.listdir(dst_root_msk) if f.lower().endswith(".png")]

for fname in mask_filenames:
    mask_path      = os.path.join(dst_root_msk, fname)
    stretched_path = os.path.join(dst_root_str, fname)

    # Skip if the stretched counterpart does not exist
    if not os.path.exists(stretched_path):
        print(f"⚠️  Skipping {fname}: stretched file not found.")
        continue

    # ---- read files as grayscale ----
    mask_img      = cv2.imread(mask_path,      cv2.IMREAD_GRAYSCALE)
    stretched_img = cv2.imread(stretched_path, cv2.IMREAD_GRAYSCALE)

    if mask_img is None or stretched_img is None:
        print(f"⚠️  Skipping {fname}: unable to read one of the images.")
        continue

    # ---- binarise mask to {0,1} ----
    _, mask_bin = cv2.threshold(mask_img, 127, 1, cv2.THRESH_BINARY)

    # ---- zero out star pixels ----
    stretched_masked = stretched_img.copy()
    stretched_masked[mask_bin == 1] = 0

    # ---- write to new folder ----
    out_path = os.path.join(dst_root_smk, fname)
    cv2.imwrite(out_path, stretched_masked)

print("✅  All stretched-masked images have been created in",
      dst_root_smk)